# debug web
#https://pythononpow.medium.com/debugging-a-tornado-pythononwheels-web-application-with-vscode-d9f5c5e5603f

In [34]:
# {
#     // 使用 IntelliSense 了解相关属性。 
#     // 悬停以查看现有属性的描述。
#     // 欲了解更多信息，请访问: https://go.microsoft.com/fwlink/?linkid=830387
#     "version": "0.2.0",
#     "configurations": [
        
#         {  
#             "name": "Python: PoW",  
#             "type": "python",  
#             "request": "launch",  
#             "program": "${workspaceFolder}/instock/web/web_service.py",  
#             "args": [],  
#             "console": "integratedTerminal"  
#         }
            
#     ]
# }

In [35]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime

import sys

# stdout=sys.stdout
sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd

# 初始化db

In [36]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}



## def func

In [37]:
def write_code_to_blk(codew, blk='060'):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
    if write_blk == 'y' or write_blk == 'Y':
        hdf5_wri = cct.cct_raw_input(
            "Rewrite code [Y] or append [N](defalut:N):")
        if hdf5_wri == 'y' or hdf5_wri == 'Y':
            append_status = False
        else:
            append_status = True
        if len(codew) > 3:
            cct.write_to_blocknew(block_path, codew, append_status,
                                  doubleFile=False, keep_last=0)
            print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write No:%s block_path:%s" % (len(codew), block_path))
            # ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

#allday
def groupby_count_code(df):
    df['couts'] = df.groupby('code')['code'].transform('count')
    if 'date' in df.columns:
        df = df.sort_values(by=['couts','date'],ascending=[0,1])
    else:
        df = df.sort_values(by=['couts'],ascending=[0])
    return df

In [38]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

#前一日
def day_last_days(daynow,last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

#上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result
    
# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0



In [39]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
        logging.error("执行信息：数据库不存在，将创建。")

1

## 放量上涨table 
cn_stock_strategy_entercn_stock_strategy_enter

In [40]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


def panda_df(conn,sql):
    df = pd.read_sql(sql, conn)
    print(df.shape)
    return df

from pandas import DataFrame as df
def exe_sql_select(conn,sql):
	cur = conn.cursor() 
	data = None
	try: # 使用异常处理，以防程序无法正常运行
		cur.execute(sql) 
		data = df(cur.fetchall(), columns = [col[0] for col in cur.description]) 
	except Exception as e:
		conn.rollback() # 发生错误时回滚
		print('事务处理失败', e)
	else:
		# conn.commit() # 事务提交
		print('事务处理成功', cur.rowcount)
	# cur.close()
	return data

def do_select_db(sql):
    conn = pymysql.connect(**MYSQL_CONN_DBAPI)
    mycursor = conn.cursor()
    print(sql)
    df = panda_df(conn,sql)
    conn.close()
    return df

### 读取SQL数据

In [41]:
conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcolall = f''' `date`,`code`,`name`,`rate_1` '''
_selcol = f''' `code`,`name`,`rate_1` '''

_table_name = 'cn_stock_strategy_enter'

lastday = last_tddate(1)

today_now = last_tddate(0)



sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{today_now}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' AND rate_1 > 0 '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' '''


sql_now_last7day = f'''SELECT {_selcolall} FROM `{_table_name}` WHERE `date` >= '{last_tddate(7)}' '''

sql_now_alltoday = f'''SELECT {_selcolall} FROM `{_table_name}`  '''

sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
print(sql_now_today)

mycursor = conn.cursor()
# mycursor.execute(sql_now_price)
# print(_selcol)
# count = 0
# for field in mycursor.fetchall():
# 		# print(field[0],end='  ')
# 		# print(field[1:4])
# 		# print(field)
# 		count +=1
# print("all:%s"%(count))


#lastday 
# df = exe_sql_select(conn,sql_now_price)

#today:
df = exe_sql_select(conn,sql_now_today)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn,sql_now_today)

# df3all = panda_df(conn,sql_now_alltoday)

dflast7d = panda_df(conn,sql_now_last7day)

# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
	
conn.close()

SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2024-03-01' AND rate_1 > 0 
SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2024-03-02' 
事务处理成功 0
(0, 3)
(40, 4)


In [42]:
#today
print(df2[:2],df2.shape)
vol_code = df2.code.tolist()[:2]
#allday
df2[:2]
dflast7d[:2]


Empty DataFrame
Columns: [code, name, rate_1]
Index: [] (0, 3)


,code,name,rate_1


,date,code,name,rate_1
0,2024-02-22,000983,山西焦煤,-3.94
1,2024-02-22,301030,仕净科技,1.18


In [43]:
# df3all = groupby_count_code(df3all)
dflast7d = groupby_count_code(dflast7d)

### show couts

In [44]:
filter_date = '2023-04-28'
filter_date = today_now
# last_tddate(0)
# filter_date = lastday

In [45]:
df7multiIndex = dflast7d.reset_index().set_index(['code','date'])
df7multiIndex[:5]
df7tail = df7multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])

index  name  rate_1  couts
code   date                                  
600938 2024-02-22      3  中国海油   -1.23      2
601101 2024-02-22      5  昊华能源    3.04      2
       2024-02-23     11  昊华能源   -4.87      2
000550 2024-02-26     15  江铃汽车    3.97      2
000921 2024-02-26     16  海信家电    7.07      2

### show Today

In [46]:
df7tail =  df7tail.reset_index().set_index('code')
# df7tail[ df7tail.rate_1 ] 
df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())][:6]

,date,index,name,rate_1,couts
code,,,,,


In [47]:
df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ][:10]

,date,index,name,rate_1,couts
code,,,,,


### 查询macd

In [48]:
# sql_select = '''select * FROM cn_stock_indicators WHERE code='000001' ORDER BY date desc limit 5'''
sql_select_code = f'''select date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,boll_ub,boll FROM cn_stock_indicators WHERE macd>0 AND macds >0 AND macdh >0 AND date=CURDATE() AND close >=boll_ub '''
df_macd_boll=do_select_db(sql_select_code)

select date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,boll_ub,boll FROM cn_stock_indicators WHERE macd>0 AND macds >0 AND macdh >0 AND date=CURDATE() AND close >=boll_ub 
(0, 12)


In [49]:
# df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])
# df7tail
df_macd_boll.shape

(0, 12)

#### 写入065 df_macd_boll 

In [50]:
if (len(df_macd_boll) > 0):
    write_code_to_blk(df_macd_boll.code.tolist(),'065')

### 写入放量上涨061

In [51]:
# df7code = df7tail[df7tail.couts >1].index.get_level_values('code')
# filter_date = '2023-04-28' couts >1

df7code = df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())].index

#nowday
df7code = df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ].index
df7code
len(df7code)

Index([], dtype='object', name='code')

0

In [52]:

# hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
# if len(hdf5_wri) == 0:
#     pass
# elif hdf5_wri == 'y' or hdf5_wri == 'Y':
#     append_status = False
# else:
#     append_status = True
# df2.code.values
#today:
# write_code_to_blk(df7code,'061')

# write_code_to_blk(df7code,'062')

# write_code_to_blk(df7code,'064')


## 读取cn_stock_strategy_keep_increasing

In [53]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_strategy_keep_increasing'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = '2023-05-08'
start_d = last_tddate(5)
# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` > '{today_now}'- INTERVAL 5 DAY '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df_increas = panda_df(engine(),sql_now_today)

df_increasA = panda_df(engine(),sql_now_allday)

2024-03-02
(0, 103)
(0, 103)


In [54]:
df_increas = df_increas.sort_values(by=['rate_1'],ascending=[1])
df_increas

,date,code,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,...,rate_91,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100


## 写入周063

In [55]:
# write_code_to_blk(df_increas.code.tolist(),'063')

### Filter Day多日

In [56]:
df_increas.shape
df_increasA.shape
df_increasA.columns[:10]
# df_iA = df_increasA.groupby(level=[0]).tail(1)
df_iA = groupby_count_code(df_increas)
df_iA_multiIndex = df_iA.reset_index().set_index(['code','date'])
df_iA_multiIndex.shape
# df_iA_multiIndex[:10]
# df_iA_multiIndex[-10:]
# df_iA.loc[df_iA.code == '000028']
df_iA_multiIndex[df_iA_multiIndex.index.get_level_values('code') == '000028'][:2]

(0, 103)

(0, 103)

Index(['date', 'code', 'name', 'rate_1', 'rate_2', 'rate_3', 'rate_4',
       'rate_5', 'rate_6', 'rate_7'],
      dtype='object')

(0, 103)

,,index,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,rate_8,...,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100,couts
code,date,,,,,,,,,,,,,,,,,,,,,


In [57]:
df_iAtail = df_iA_multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df_iAtail =  df_iAtail.reset_index().set_index('code')
df_iAtail.loc[:,['date','name','couts']][:5]
# df_iA.loc[:,['date','code','name','couts']][:50]

# df_iAtail[df_iAtail.couts < 3].loc[:,['date','name','couts']]

,date,name,couts
code,,,


### show Couts

In [58]:
df_iA.loc[:,['date','code','name','couts']]

,date,code,name,couts


## cn_stock_indicators MutilIndex

### 读取SQL数据表

In [59]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_indicators'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = last_tddate(5)

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(engine(),sql_now_today)

# df2 = panda_df(engine(),sql_now_allday)



# conn.close()

2024-03-02
(9065, 77)


In [60]:
df2.columns
df2.loc[df2.code =='000061'][:5]

Index(['date', 'code', 'name', 'close', 'macd', 'macds', 'macdh', 'kdjk',
       'kdjd', 'kdjj', 'boll_ub', 'boll', 'boll_lb', 'trix', 'trix_20_sma',
       'tema', 'cr', 'cr-ma1', 'cr-ma2', 'cr-ma3', 'rsi_6', 'rsi_12', 'rsi',
       'rsi_24', 'vr', 'vr_6_sma', 'roc', 'rocma', 'rocema', 'pdi', 'mdi',
       'dx', 'adx', 'adxr', 'wr_6', 'wr_10', 'wr_14', 'cci', 'cci_84', 'tr',
       'atr', 'dma', 'dma_10_sma', 'obv', 'sar', 'psy', 'psyma', 'br', 'ar',
       'emv', 'emva', 'bias', 'mfi', 'mfisma', 'vwma', 'mvwma', 'ppo', 'ppos',
       'ppoh', 'wt1', 'wt2', 'supertrend_ub', 'supertrend', 'supertrend_lb',
       'dpo', 'madpo', 'vhf', 'rvi', 'rvis', 'fi', 'force_2', 'force_13',
       'ene_ue', 'ene', 'ene_le', 'stochrsi_k', 'stochrsi_d'],
      dtype='object')

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
42,2024-02-26,000061,农 产 品,6.12,-0.015983,-0.081396,0.065413,73.8275,70.5725,80.3375,...,0.129517,0.082859,-263404.0,-231622.0,429824.0,6.64335,6.04485,5.44635,91.1199,95.1111
4574,2024-02-27,000061,农 产 品,6.20,-0.002683,-0.065629,0.062946,71.4406,70.8618,72.5980,...,0.176737,0.116600,606768.0,327305.0,455101.0,6.75768,6.14888,5.54008,98.6908,94.9870


In [61]:
df2[ (df2.close >= df2.boll_ub) & (df2.close * 0.95  <= df2.boll_ub) & (df2.macdh > 0)& (df2.macd > 0)& (df2.macds > 0)]
# .loc[:,['rsi_6','rsi_12','rsi_24','rsi']]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
391,2024-02-26,000951,中国重汽,16.76,0.584187,0.349233,0.234954,87.6324,82.7343,97.4287,...,0.265134,0.168885,12804300.0,10462600.0,5734330.0,16.95410,15.42670,13.89930,100.0000,100.0000
601,2024-02-26,002085,万丰奥威,7.05,0.406886,0.261135,0.145751,88.8473,81.0870,104.3680,...,0.113740,0.060840,52705600.0,40207500.0,19532300.0,6.75324,6.14484,5.53644,100.0000,99.5267
1165,2024-02-26,002673,西部证券,6.90,0.130261,0.024208,0.106053,80.1057,72.9881,94.3409,...,0.232926,0.125441,-24054300.0,18592200.0,20244300.0,7.05738,6.42158,5.78578,92.6081,94.3538
1420,2024-02-26,002959,小熊电器,56.25,0.722617,0.079079,0.643538,86.9470,81.1057,98.6297,...,0.279157,0.248332,33682100.0,22548900.0,5156440.0,56.82090,51.70190,46.58290,100.0000,87.5169
1832,2024-02-26,300353,东土科技,10.61,0.298101,0.073693,0.224408,85.6391,77.5335,101.8500,...,0.299106,0.262454,36801900.0,27941600.0,12043800.0,10.56940,9.61722,8.66502,100.0000,94.1220
2480,2024-02-26,301030,仕净科技,50.88,2.379840,1.609490,0.770347,81.8429,78.9058,87.7171,...,0.180148,0.181705,2044880.0,5486930.0,7378410.0,50.87020,46.28730,41.70440,100.0000,100.0000
3282,2024-02-26,600582,天地科技,6.64,0.201965,0.126334,0.075631,88.5605,84.1825,97.3164,...,0.248715,0.180520,26509400.0,19193300.0,6603310.0,6.71772,6.11252,5.50732,100.0000,100.0000
3654,2024-02-26,601107,四川成渝,5.00,0.111105,0.036209,0.074895,82.0304,77.4543,91.1825,...,0.217059,0.120641,-7439090.0,-657096.0,1749530.0,5.04717,4.59247,4.13777,82.8386,94.2795
3731,2024-02-26,601567,三星医疗,25.39,0.991675,0.711462,0.280213,83.6392,78.7347,93.4482,...,0.293568,0.263133,28051300.0,18979600.0,6483050.0,25.77530,23.45320,21.13110,100.0000,92.4794
3954,2024-02-26,603135,中重科技,18.60,0.719207,0.148208,0.570998,83.0933,81.6653,85.9495,...,0.368890,0.251320,-10279400.0,3775250.0,6367950.0,18.05080,16.42460,14.79840,96.3409,98.0970


# inStock 序列化数据测试

## 定义参数

In [62]:
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\2020'+get_today(sep='')[-4:-2]+'\\2020'+get_today(sep='')[-4:]
stock_hist_cache_path

'Y:\\dockerf\\stock\\instockcache\\hist\\202003\\20200302'

In [63]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
get_today()
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\2020'+get_today(sep='')[-4:]
code = '601336'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


'2024-03-02'

Y:\dockerf\stock\instockcache\hist\202003\20200302\601336qfq.gzip.pickle


FileNotFoundError: [Errno 2] No such file or directory: 'Y:\\dockerf\\stock\\instockcache\\hist\\202003\\20200302\\601336qfq.gzip.pickle'

In [ ]:
df.shape
df[-2:]
# df.columns
lastday=df.date.values[-1]
lastday
data = df.copy()
# macd
data.loc[:, 'diff'], data.loc[:, 'dea'], data.loc[:, 'macd'] = tl.MACD(
    data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['diff'].values[np.isnan(data['diff'].values)] = 0.0
data['dea'].values[np.isnan(data['dea'].values)] = 0.0
data['macd'].values[np.isnan(data['macd'].values)] = 0.0

data.loc[:, 'boll_ub'], data.loc[:, 'boll'], data.loc[:, 'boll_lb'] = tl.BBANDS \
    (data['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['boll_ub'].values[np.isnan(data['boll_ub'].values)] = 0.0
data['boll'].values[np.isnan(data['boll'].values)] = 0.0
data['boll_lb'].values[np.isnan(data['boll_lb'].values)] = 0.0
data.iloc[-60:]['macd'].max()
# DIFF:=EMA(CLOSE,12) - EMA(CLOSE,26);  256 
# DEA := EMA(DIFF,9); 1.52
# MACD := 2*(DIFF-DEA); 2.08
print(not False)

## stock_period_to_typ Resample 'W-FRI'

In [ ]:
def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).first()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()
    
    lastWeek1 = str(period_stock_data['open'].index.values[-1])[:10]
    lastweek2 = str(period_stock_data['open'].index.values[-2])[:10]
    if ratiodays:
        if period_day == 'W-FRI':
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
            ratio_d =(5-(duratio%5))/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day.find('W') >= 0:
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastday, '%Y-%m-%d').date() - datetime.datetime.strptime(lastweek2, '%Y-%m-%d').date())[0])
            ratio_d =(duratio)/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day == 'BM':
            # daynow = '2023-04-26'
            # lastday = '2023-04-23'
            print(lastWeek1,lastweek2,lastday,lastday2)
            # print((str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2]))
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2])
            ratio_d =(30-(duratio%30))/30
            print("ratio_d:%s %s dura:%s"%(ratio_d,lastday,duratio))
        elif period_day.find('M') >= 0:
            ratio_d = 1
            
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays and ratio_d != 1:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    if not indextype and period_stock_data.index.name == 'date':
        # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
        period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
        period_stock_data.index.name = 'date'
    else:
        if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    # print period_stock_data
    return period_stock_data


### Resample Week W-FRI

In [ ]:
# dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
# dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
# data[-5:]

### Resample Month BM
https://note.nkmk.me/python-pandas-time-series-freq/

    BM: 月末最終営業日ごと
    MS: 毎月の開始日ごと
    BMS: 毎月の最初の営業日ごと


In [ ]:
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-2:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]

In [ ]:
daynow=dfw.index.values[-1]
str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


# inStock RSI

In [ ]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")

In [68]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday

(0, 3)

,code,name,rate_1


,code,name,rate_1


AttributeError: 'DataFrame' object has no attribute 'date'

In [67]:
data=df.copy()

In [65]:
def check_rsi_status(df,threshold=60,period_day=False):
    # macd
    data = df.copy()
    # cci 计算方法和结果和stockstats不同，stockstats典型价采用均价(总额/成交量)计算
    data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
    data['cci'].values[np.isnan(data['cci'].values)] = 0.0
    # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
    # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
    rsilast5 = data.iloc[-6:-1]['cci']
    rsilast5max = rsilast5.max()
    rsilast = data.iloc[-1]['cci']

    if rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max:
        return True
    else:
        return False

In [66]:
# data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
# data['cci'].values[np.isnan(data['cci'].values)] = 0.0
# # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
# # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
# rsilast5 = data.iloc[-6:-1]['cci']
# rsilast5max = rsilast5.max()
# rsilast = data.iloc[-1]['cci']
# data['cci'][-6:]
# rsilast5.iloc[-1]
# rsilast5,rsilast
# rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max
check_rsi_status(df)

KeyError: 'high'